In [1]:
import os
os.chdir("/work/pi_dhruveshpate_umass_edu/aamballa_umass_edu/SuperposedDecoding/")

In [2]:
%load_ext autoreload
%autoreload 2

import json
import os
import pickle
from datetime import datetime

# import evaluate
import torch
from tqdm import tqdm

from eval import *
from superposed.llama.metrics import *
from superposed.llama.generation import Llama
from superposed.llama.superposed_generation import SuperposedLlama
from superposed.llama.tokenizer import Tokenizer
from superposed.ngrams.ngram_models import make_models

In [4]:
sup_device = torch.device("cuda:0")

In [5]:
# Params
param_file = "params/p15_d3_mixed.json"
with open(param_file, "r") as f:
    params = json.load(f)
    print(f"Parameters: {params}")
alpha = params["alpha"]
temp = params["temp"]
n_drafts = params["n_drafts"]
prompt_len = params["prompt_len"]
n_token_sample = params["n_token_sample"]
i_weights = params["i_weights"]
i_length = params["i_length"]

Parameters: {'alpha': 0.54, 'temp': 0.06, 'n_drafts': 3, 'prompt_len': 15, 'n_token_sample': 9, 'n_token_consider': 32000, 'mixing_method': 'sample_new_weights_with_score', 'smoothing': 'geom', 'sample_tokens': 0, 'sample_beams': 0, 'i_weights': [0.01, 0.04, 0.15, 0.18, 0.12], 'i_length': [1, 2, 3, 4, 5]}


In [ ]:
# Create ngram models
ngrams = make_models("ngrams/ckpts-400k", bigram=True, trigram=True, fourgram=True, fivegram=True, sixgram=True, sevengram=False)

Making bigram...
1310808
Making trigram...
335544408
Making fourgram...
1342177368
Making fivegram...


In [6]:
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "127.0.0.1"
os.environ["MASTER_PORT"] = "10301"

In [7]:
weight_path = "7B/llama-2-7b"
token_path = "7B/"

# loading transformer model (suitable for superposed generaeion) from checkooint
model = SuperposedLlama.build(ckpt_dir=weight_path, 
                         tokenizer_path=f'{token_path}/tokenizer.model', 
                         max_seq_len=100, 
                         max_batch_size=32,
                         device=sup_device,
                         model_parallel_size=1)

tokenizer = Tokenizer('7B/tokenizer.model')

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/work/pi_dhruveshpate_umass_edu/aamballa_umass_edu/.conda/envs/Superposed/lib/python3.9/site-packages/torch/__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


Loaded in 16.90 seconds
cuda:0


# Inference

In [8]:
def decode(tokenizer, encoding):
    """
    Args:
        tokenizer (Any): Tokenizer
        encoding (torch.Tensor): Encoding
    Returns:
        decoding (str)
    """
    eos_locs = (encoding == tokenizer.eos_id).nonzero()
    if len(eos_locs > 0):
        encoding = encoding[:eos_locs[0]]
    return tokenizer.decode(encoding.to(torch.int32).tolist())

In [9]:
prompts = [
    "Hi my name is",
    "The Seattle Seahawks were Super Bowl",
    "Penguins are birds native to"
]
tokenized_prompts = tokenizer.encode(prompts, True, False)

In [19]:
alive_gens, token_probs, ngram_prob_mtx = model.sup_generate(prompt_tokens=tokenized_prompts, 
                                        smoothing="geom",
                                        max_gen_len=10, 
                                        n_token_sample=n_token_sample,
                                        alpha=alpha, 
                                        temp=temp,
                                        n_drafts=n_drafts,
                                        i_weights=i_weights,
                                        i_length=i_length,
                                        ngrams=ngrams,
                                        get_time=False,
                                        penalty=200,
                                        get_model_probs=True)

NameError: name 'model' is not defined

In [11]:
alive_gens_no_ngram, _ = model.sup_generate(prompt_tokens=tokenized_prompts, 
                                        smoothing=None,
                                        max_gen_len=10, 
                                        n_token_sample=n_token_sample,
                                        alpha=alpha, 
                                        temp=temp,
                                        n_drafts=n_drafts,
                                        i_weights=i_weights,
                                        i_length=i_length,
                                        ngrams=[],
                                        get_time=False,
                                        penalty=200,
                                        get_model_probs=False)

In [13]:
gens = alive_gens_no_ngram[0].reshape(len(prompts) * n_drafts, -1)
for i in gens:
    print(decode(tokenizer, i))

# GSM 8K 

In [18]:
GSM8K_EXEMPLARS = [
    {
        "question": "There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
        "reasoning": "There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.",
        "answer": "6",
    },
    {
        "question": "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?",
        "reasoning": "There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.",
        "answer": "5",
    },
    {
        "question": "Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
        "reasoning": "Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39.",
        "answer": "39",
    },
    {
        "question": "Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
        "reasoning": "Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8.",
        "answer": "8",
    },
    {
        "question": "Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?",
        "reasoning": "Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9.",
        "answer": "9",
    },
    {
        "question": "There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?",
        "reasoning": "There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29.",
        "answer": "29",
    },
    {
        "question": "Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?",
        "reasoning": "Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls.",
        "answer": "33",
    },
    {
        "question": "Olivia has $23. She bought five bagels for $3 each. How much money does she have left?",
        "reasoning": "Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8.",
        "answer": "8",
    },
]

prompts = [GSM8K_EXEMPLARS[i]["question"] for i in range(len(GSM8K_EXEMPLARS))]
tokenized_prompts = tokenizer.encode(prompts, True, False)

In [19]:
alive_gens_no_ngram, _ = model.sup_generate(prompt_tokens=tokenized_prompts, 
                                        smoothing=None,
                                        max_gen_len=10, 
                                        n_token_sample=n_token_sample,
                                        alpha=alpha, 
                                        temp=temp,
                                        n_drafts=n_drafts,
                                        i_weights=i_weights,
                                        i_length=i_length,
                                        ngrams=[],
                                        get_time=False,
                                        penalty=200,
                                        get_model_probs=False)

In [20]:
gens = alive_gens_no_ngram[0].reshape(len(prompts) * n_drafts, -1)
for i in gens:
    print(decode(tokenizer, i))

There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
There are 15 trees in the gro
There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
The are 15 trees in the gro
There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A are 15 trees in the gro
If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
How many cars are in the parking lot?
How many cars are in the parking lot if there are 3 cars in the
If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
How many cars are in the parking lot?
If many cars are in the parking lot i